# APIM ❤️ OpenAI

## モデルルーティングラボ
![flow](../../images/model-routing.gif)

Azure OpenAIモデルとバージョンに基づいてバックエンドにルーティングするためのプレイグラウンド。

### 目次
- [0️⃣ ノートブック変数の初期化](#0)
- [1️⃣ Azureリソースグループの作成](#1)
- [2️⃣ 🦾 Bicepを使用したデプロイの作成](#2)
- [3️⃣ デプロイの出力を取得](#3)
- [🧪 直接HTTPコールを使用したAPIのテスト](#requests)
- [🧪 Azure OpenAI Python SDKを使用したAPIのテスト](#sdk)
- [🗑️ リソースのクリーンアップ](#clean)

### バックログ
- ノートブックの改善

### 前提条件
- [Python 3.8以降のバージョン](https://www.python.org/)がインストールされていること
- [Pandasライブラリ](https://pandas.pydata.org/)がインストールされていること
- [VS Code](https://code.visualstudio.com/)がインストールされ、[Jupyterノートブック拡張機能](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)が有効になっていること
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli)がインストールされていること
- Contributor権限を持つ[Azureサブスクリプション](https://azure.microsoft.com/en-us/free/)があること
- [Azure OpenAIへのアクセス](https://aka.ms/oai/access)が許可されていること
- [Azure CLIでAzureにサインイン](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)していること

<a id='0'></a>
### 0️⃣ ノートブック変数の初期化

- リソースはサブスクリプションIDに基づいた一意の文字列でサフィックスされます
- ロケーションパラメータは、[Azureリージョンごとの製品の利用可能性](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management)に基づいて調整してください
- OpenAIモデルとバージョンは、[リージョンごとの利用可能性](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models)に基づいて調整してください

In [ ]:
import os
import json
import datetime
import requests

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
index = ''  # Set a matching value here and in clean-up-resources.ipynb if you want separate instances of the lab. This is helpful when tearing down resources and mitigating API Management's soft delete.
resource_group_name = f"lab-{deployment_name}{index}" # change the name to match your naming style
resource_group_location = "westeurope"

# Define three OpenAI model and version combinations
# https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#gpt-35
# Please note that availability of models and versions is variable and that you may need to adjust the model and version names to match the available models and versions in your Azure subscription.
# For this lab, we are using the following combinations based on PayGo availability on June 21, 2024:
#
#   1) GPT-3.5 Turbo 1106: France Central, Sweden Central
#   2) GPT-3.5 Turbo 0125: North Central US, South Central US
#   3) GPT-4o 2024-05-13: East US, West US

openai_model_gpt_35_turbo_0125 = { "name": "gpt-35-turbo", "version": "0125" }
openai_model_gpt_35_turbo_1106 = { "name": "gpt-35-turbo", "version": "1106" }
openai_model_gpt_4o_20240513 = { "name": "gpt-4o", "version": "2024-05-13" }

openai_model_1_name = "gpt-35-turbo"
openai_model_1_version = "1106"
openai_deployment_1_name = f"{openai_model_1_name}-{openai_model_1_version}"
openai_resources_1 = [ {"name": "oai-francecentral", "location": "francecentral"}, {"name": "oai-swedencentral", "location": "swedencentral"} ]

openai_model_2_name = "gpt-35-turbo"
openai_model_2_version = "0125"
openai_deployment_2_name = f"{openai_model_2_name}-{openai_model_2_version}"
openai_resources_2 = [ {"name": "oai-northcentralus", "location": "northcentralus"}, {"name": "oai-southcentralus", "location": "southcentralus"} ]

openai_model_3_name = "gpt-4o"
openai_model_3_version = "2024-05-13"
openai_deployment_3_name = f"{openai_model_3_name}-{openai_model_3_version}"
openai_resources_3 = [ {"name": "oai-eastus", "location": "eastus"}, {"name": "oai-westus", "location": "westus"} ]

# Define Azure OpenAI resources
openai_resources_sku = "S0"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'

# Define Azure API Management
apim_resource_name = "apim"
apim_resource_location = "westeurope"
apim_resource_sku = "Basicv2"

# Define the Azure OpenAI backends and backend pools per Azure OpenAI model and version
openai_backend_pool_1 = f"oai-backend-pool-{openai_deployment_1_name}"
openai_backend_pool_2 = f"oai-backend-pool-{openai_deployment_2_name}"
openai_backend_pool_3 = f"oai-backend-pool-{openai_deployment_3_name}"

log_analytics_name = "workspace"
app_insights_name = 'insights'


<a id='1'></a>
### 1️⃣ Azureリソースグループの作成
このラボでデプロイされるすべてのリソースは、指定されたリソースグループに作成されます。既存のリソースグループを使用する場合は、このステップをスキップしてください。

In [ ]:
# type: ignore
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location} 

if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())

<a id='2'></a>
### 2️⃣ 🦾 Bicepを使用したデプロイの作成

このラボでは、[Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep)を使用して、デプロイされるすべてのリソースを宣言的に定義します。異なる構成を試すために、パラメータや[main.bicep](main.bicep)を直接変更してください。

In [ ]:
if len(openai_resources_1) > 0:
    backend_id_1 = openai_backend_pool_1 if len(openai_resources_1) > 1 else openai_resources_1[0].get("name")
if len(openai_resources_2) > 0:
    backend_id_2 = openai_backend_pool_2 if len(openai_resources_2) > 1 else openai_resources_2[0].get("name")
if len(openai_resources_3) > 0:
    backend_id_3 = openai_backend_pool_3 if len(openai_resources_3) > 1 else openai_resources_3[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    policy_xml = policy_template_xml.replace("{backend-id-1}", backend_id_1).replace("{backend-id-2}", backend_id_2).replace("{backend-id-3}", backend_id_3)
    policy_xml_file.close()
open("policy.xml", 'w').write(policy_xml)

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "openAIBackendPoolName_1": { "value": openai_backend_pool_1 },
    "openAIBackendPoolName_2": { "value": openai_backend_pool_2 },
    "openAIBackendPoolName_3": { "value": openai_backend_pool_3 },
    "openAIConfig_1": { "value": openai_resources_1 },
    "openAIConfig_2": { "value": openai_resources_2 },
    "openAIConfig_3": { "value": openai_resources_3 },
    "openAIDeploymentName_1": { "value": openai_deployment_1_name },
    "openAIDeploymentName_2": { "value": openai_deployment_2_name },
    "openAIDeploymentName_3": { "value": openai_deployment_3_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName_1": { "value": openai_model_1_name },
    "openAIModelName_2": { "value": openai_model_2_name },
    "openAIModelName_3": { "value": openai_model_3_name },
    "openAIModelVersion_1": { "value": openai_model_1_version },
    "openAIModelVersion_2": { "value": openai_model_2_version },
    "openAIModelVersion_3": { "value": openai_model_3_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku},
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name },
    "index": { "value": index}
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

open("policy.xml", 'w').write(policy_template_xml)


<a id='3'></a>
### 3️⃣ デプロイの出力を取得

テストの準備が整う前に、ゲートウェイURLとサブスクリプションを取得する必要があります。

In [ ]:
# type: ignore
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = deployment_stdout.n

# type: ignore
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = deployment_stdout.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)

# type: ignore
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.logAnalyticsWorkspaceId.value -o tsv
workspace_id = deployment_stdout.n
print("👉🏻 Workspace ID: ", workspace_id)

# type: ignore
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.applicationInsightsAppId.value -o tsv
app_id = deployment_stdout.n
print("👉🏻 App ID: ", app_id)



<a id='requests'></a>
### 🧪 直接HTTPコールを使用したAPIのテスト
Requestsは、ここで生のAPIリクエストを行い、レスポンスを検査するために使用される、エレガントでシンプルなPython用のHTTPライブラリです。

In [ ]:
import time
runs = 2
sleep_time_ms = 1000

# Initialize a session for connection pooling
session = requests.Session()

def make_api_request(deployment_name, openai_resources, apim_subscription_key, apim_resource_gateway_url, openai_api_version, sleep_time_ms):
    if len(openai_resources) > 0:
        messages = {
            "messages": [
                {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
                {"role": "user", "content": "Can you tell me the time, please?"}
            ]
        }
        url = f"{apim_resource_gateway_url}/openai/deployments/{deployment_name}/chat/completions?api-version={openai_api_version}"
        response = session.post(url, headers={'api-key': apim_subscription_key}, json=messages)
        print("url: ", url)
        print("status code: ", response.status_code)
        print("headers ", response.headers)
        print("x-ms-region: ", response.headers.get("x-ms-region"))  # Useful to determine the region of the backend
        if response.status_code == 200:
            data = response.json()
            print("\nresponse: ", data.get("choices")[0].get("message").get("content"))
        else:
            print(response.text)
        time.sleep(sleep_time_ms / 1000)

# Define your deployments and resources
deployments = [
    {"name": openai_deployment_1_name, "resources": openai_resources_1},    # GPT-3.5 Turbo 1106
    {"name": openai_deployment_2_name, "resources": openai_resources_2},    # GPT-3.5 Turbo 0125
    {"name": openai_deployment_3_name, "resources": openai_resources_3},    # GPT-4o 2024-05-13
]

# Loop through each deployment and make the API request
for deployment in deployments:
    for i in range(runs):
        print(f"\n▶️ Run: {i+1} for deployment {deployment['name']}")
        make_api_request(deployment['name'], deployment['resources'], apim_subscription_key, apim_resource_gateway_url, openai_api_version, sleep_time_ms)
        

<a id='sdk'></a>
### 🧪 Azure OpenAI Python SDKを使用したAPIのテスト
OpenAPIは広く使用されている[Pythonライブラリ](https://github.com/openai/openai-python)を提供しています。このライブラリには、すべてのリクエストパラメータとレスポンスフィールドの型定義が含まれています。このテストの目的は、APIMがOpenAIへのリクエストをシームレスにプロキシし、その機能を妨げることなく動作することを確認することです。
- 注意: このステップを実行する前に、ターミナルで```pip install openai```を実行してください。

In [ ]:
import time
runs = 2
sleep_time_ms = 1000

def make_openaisdk_request(deployment_name, openai_resources, apim_subscription_key, apim_resource_gateway_url, openai_api_version, sleep_time_ms):
    from openai import AzureOpenAI
    if len(openai_resources) > 0:
        messages = {
            "messages": [
                {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
                {"role": "user", "content": "Can you tell me the time, please?"}
            ]
        }
        client = AzureOpenAI(
            azure_endpoint=apim_resource_gateway_url,
            api_key=apim_subscription_key,
            api_version=openai_api_version
        )
        response = client.chat.completions.create(model=openai_deployment_1_name, messages=messages)
        print(response.choices[0].message.content)
        time.sleep(sleep_time_ms/1000)

# Define your deployments and resources
deployments = [
    {"name": openai_deployment_1_name, "resources": openai_resources_1},    # GPT-3.5 Turbo 1106
    {"name": openai_deployment_2_name, "resources": openai_resources_2},    # GPT-3.5 Turbo 0125
    {"name": openai_deployment_3_name, "resources": openai_resources_3},    # GPT-4o 2024-05-13
]

# Loop through each deployment and make the API request
for deployment in deployments:
    for i in range(runs):
        print(f"\n▶️ Run: {i+1} for deployment {deployment['name']}")
        make_api_request(deployment['name'], deployment['resources'], apim_subscription_key, apim_resource_gateway_url, openai_api_version, sleep_time_ms)     


<a id='portal'></a>
### 🔍 Azureポータルでワークブックを開く

ワークブックリソースを開き、使用状況分析を確認して、モデルルーティングやその他のメトリクスを確認します。

<a id='clean'></a>
### 🗑️ リソースのクリーンアップ

ラボが終了したら、追加の料金を避け、Azureサブスクリプションを整理するために、デプロイしたすべてのリソースをAzureから削除する必要があります。
そのためには、[リソースのクリーンアップノートブック](clean-up-resources.ipynb)を使用してください。